In [1]:
import os
%pwd

'd:\\mlflow\\datascience-e2e-project1\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'd:\\mlflow\\datascience-e2e-project1'

In [6]:
from dataclasses import dataclass
from pathlib import Path

In [7]:
@dataclass
class ModelTrainingConfig:
  root_dir: Path
  train_data_path: Path
  test_data_path: Path
  model_name: str
  alpha: float
  l1_ratio: float
  target_col: str

In [10]:
from src.ds_e2e_project1.constants import *
from src.ds_e2e_project1.utils.common import read_yaml, create_directories

In [17]:
class ConfigurationManager:
  def __init__(
      self,
      config_filepath = CONFIG_FILE_PATH,
      params_filepath = PARAMS_FILE_PATH,
      schema_filepath = SCHEMA_FILE_PATH
  ):
    self.config = read_yaml(config_filepath)
    self.params = read_yaml(params_filepath)
    self.schema = read_yaml(schema_filepath)

    create_directories([self.config.artifacts_root])


  def get_model_training_config(self) -> ModelTrainingConfig:
    config = self.config.model_training
    create_directories([config.root_dir])

    model_training_config = ModelTrainingConfig(
      alpha=self.params.ElasticNet.alpha,
      l1_ratio=self.params.ElasticNet.l1_ratio,
      target_col=self.schema.TARGET_COLUMN.name,
      model_name=config.model_name,
      root_dir=config.root_dir,
      test_data_path=config.test_data_path,
      train_data_path=config.train_data_path
    )
    
    return model_training_config



In [14]:
from sklearn.linear_model import ElasticNet
import pandas as pd
import os
import joblib


In [15]:
class ModelTraining:
  def __init__(self, model_training_config: ModelTrainingConfig):
    self.config = model_training_config

  def load_train_test_data(self):
    train_data = pd.read_csv(self.config.train_data_path)
    X_train = train_data.drop(columns=[self.config.target_col]).values
    y_train = train_data[self.config.target_col].values
    return X_train, y_train

  def train_ElasticNet_model(self):
    X_train, y_train = self.load_train_test_data()
    model = ElasticNet(alpha=self.config.alpha, l1_ratio=self.config.l1_ratio, random_state=42)

    model.fit(X=X_train, y=y_train)

    joblib.dump(model, filename=os.path.join(self.config.root_dir, self.config.model_name))
    

In [18]:
try:
  config_manager = ConfigurationManager()
  model_training_config = config_manager.get_model_training_config()
  model_training = ModelTraining(model_training_config)
  model_training.train_ElasticNet_model()
except Exception as e:
  raise e

[2025-04-02 22:12:20,388 : INFO : common : YAML file config\config.yaml read successfully]
[2025-04-02 22:12:20,390 : INFO : common : YAML file params.yaml read successfully]
[2025-04-02 22:12:20,393 : INFO : common : YAML file schema.yaml read successfully]
[2025-04-02 22:12:20,395 : INFO : common : Directory artifacts created]
[2025-04-02 22:12:20,397 : INFO : common : Directory artifacts/model_training created]
